# Demo 1: Single GPU Training Baseline
## Understanding GPU Memory and Basic Training Loop

**Duration**: 15-20 minutes

This notebook demonstrates:
- Setting up PyTorch with GPU
- Creating a simple CNN for CIFAR-10
- Monitoring GPU memory usage
- Establishing baseline training metrics

**Prerequisite**: GPU available, PyTorch installed

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import time
from datetime import datetime

# Check GPU availability
print(f"CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU Device: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    print(f"CUDA Version: {torch.version.cuda}")

In [ ]:

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import time
from datetime import datetime

# Check GPU availability
print(f"CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU Device: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    print(f"CUDA Version: {torch.version.cuda}")
# Set random seeds for reproducibility
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)

# Device selection
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Training on: {device}")
# Define a simple CNN
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(256 * 4 * 4, 512)
        self.fc2 = nn.Linear(512, 10)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.pool(self.relu(self.conv3(x)))
        x = x.view(x.size(0), -1)
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

# Initialize model
model = SimpleCNN().to(device)
print(f"Model Parameters: {sum(p.numel() for p in model.parameters()):,}")
print(f"Model on device: {next(model.parameters()).device}")
# Data loading with augmentation
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Download CIFAR-10 dataset
print("Downloading CIFAR-10 dataset...")
trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

# DataLoaders
batch_size = 128
train_loader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
test_loader = DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

print(f"Training samples: {len(trainset)}, Test samples: {len(testset)}")
print(f"Batch size: {batch_size}")
# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)

# Learning rate scheduler
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)

def get_gpu_memory():
    """Get current GPU memory usage in MB"""
    if torch.cuda.is_available():
        return torch.cuda.memory_allocated() / 1024 / 1024
    return 0

print(f"Initial GPU Memory: {get_gpu_memory():.2f} MB")
# Training function
def train_epoch(epoch, train_loader, model, criterion, optimizer, device):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    start_time = time.time()
    
    for batch_idx, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)
        
        if (batch_idx + 1) % 100 == 0:
            print(f"  Batch [{batch_idx+1}/{len(train_loader)}] "
                  f"Loss: {loss.item():.4f} | "
                  f"GPU Memory: {get_gpu_memory():.2f} MB")
    
    elapsed = time.time() - start_time
    avg_loss = total_loss / len(train_loader)
    accuracy = 100 * correct / total
    
    return avg_loss, accuracy, elapsed

print("Training setup complete. Ready to train!")
# Train for a few epochs
num_epochs = 3
train_losses = []
train_accs = []
epoch_times = []

print(f"Starting training for {num_epochs} epochs...\n")
overall_start = time.time()

for epoch in range(num_epochs):
    print(f"Epoch [{epoch+1}/{num_epochs}]")
    avg_loss, accuracy, epoch_time = train_epoch(epoch, train_loader, model, criterion, optimizer, device)
    
    train_losses.append(avg_loss)
    train_accs.append(accuracy)
    epoch_times.append(epoch_time)
    
    print(f"  Loss: {avg_loss:.4f} | Accuracy: {accuracy:.2f}% | Time: {epoch_time:.2f}s")
    print(f"  GPU Memory Peak: {get_gpu_memory():.2f} MB\n")
    
    scheduler.step()

overall_time = time.time() - overall_start
print(f"\n=== Training Complete ===")
print(f"Total Time: {overall_time:.2f}s")
print(f"Average Epoch Time: {sum(epoch_times)/len(epoch_times):.2f}s")
print(f"Samples/sec: {len(trainset) * num_epochs / overall_time:.2f}")
# Visualization
import matplotlib.pyplot as plt

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

ax1.plot(train_losses, marker='o')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.set_title('Training Loss')
ax1.grid(True)

ax2.plot(train_accs, marker='o')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Accuracy (%)')
ax2.set_title('Training Accuracy')
ax2.grid(True)

plt.tight_layout()
plt.show()

print("\n=== Performance Summary ===")
print(f"Final Loss: {train_losses[-1]:.4f}")
print(f"Final Accuracy: {train_accs[-1]:.2f}%")
print(f"Max GPU Memory Used: {get_gpu_memory():.2f} MB")